## Dependencies

In [1]:
import pandas as pd
import numpy as np
import os
import shutil
from datetime import date
import csv
import time
import zipfile
import requests
import steam
from steam.steamid import SteamID
from steam.webapi import WebAPI, get
import steamleaderboards as sl
import os,sys,stat
from zipfile import ZipFile, ZipInfo

# 1. unpack stats from zipped folder

In [2]:
# user_zip = data = st.file_uploader("Upload your zipped stats folder", type=["csv", "txt"])
dataDir = 'G:\Steam\steamapps\common\FPSAimTrainer\FPSAimTrainer\stats_comp.zip'
target = ['Kills:', 'Avg TTK:', 'Score:', 'Scenario:']
temp_folder = '../temp'

# unzip and extract to a temporary folder
with zipfile.ZipFile(dataDir, 'r') as zip_ref:
   zip_ref.extractall(temp_folder) # If files with same filename are already present at extraction location then it will overwrite those files.


In [3]:

unzipped_stats = temp_folder + '\stats'

try:
    stats = pd.read_csv(unzipped_stats)
except:
    stats = pd.DataFrame(columns=['Kills', 'Avg TTK', 'Score', 'Scenario'])

In [4]:
stats

,Kills,Avg TTK,Score,Scenario


In [5]:
# Empty dict to store our csv data
data = {}

# Go through all the files and store our target data in our empty dictionary, including the datetime of the file.
for filename in os.listdir(unzipped_stats):
    
    f = open(f'{unzipped_stats}/{filename}', 'r')
    with f as csv_file:
        csv_reader = csv.reader(csv_file)
        for line in csv_reader:
            try:
                if line[0] in target:
                    data[(line[0][:-1])] = line[1]     
            except IndexError:
                continue
        f.close()

    scenario_date = f'{unzipped_stats}/{filename}'[-29:-10]
    data["Date"] = scenario_date
    df = pd.DataFrame(data, index=[0])
    stats = pd.concat([stats,df]).drop_duplicates().reset_index(drop=True)

In [6]:
df

,Kills,Avg TTK,Score,Scenario,Date
0,103,0.59223,99.149529,Wide Wall 6Targets,2022.05.22-12.42.58


#### (FROM DIRECTORY)...

In [7]:
#dataDir = 'G:\Steam\steamapps\common\FPSAimTrainer\FPSAimTrainer\stats'
#target = ['Kills:', 'Avg TTK:', 'Score:', 'Scenario:']
#try:
#    stats = pd.read_csv(dataDir + '\\stats.csv')
#except:
#    stats = pd.DataFrame(columns=['Kills', 'Avg TTK', 'Score', 'Scenario'])
#data = {}
#
#

# CLEAN


In [8]:
stats.head(3)

,Kills,Avg TTK,Score,Scenario,Date
0,2,0,10792.587891,180 Regen Warzone,2021.07.27-22.47.26
1,12,0,22690.832031,180 Regen Warzone,2021.07.27-22.48.41
2,7,0,15467.339844,180 Regen Warzone,2021.07.27-22.49.53


In [9]:
stats.isna().sum() #NO Nan

Kills       0
Avg TTK     0
Score       0
Scenario    0
Date        0
dtype: int64

In [10]:
stats.duplicated().sum() #NO duplicates

0

In [11]:
stats['Kills'] = pd.to_numeric(stats['Kills'], downcast='integer')
stats['Avg TTK'] = pd.to_numeric(stats['Avg TTK'], downcast='float')
stats['Score'] = pd.to_numeric(stats['Avg TTK'], downcast='float')
stats['Date'] = pd.to_datetime(stats['Date'], yearfirst=True, format=('%Y.%m.%d-%H.%M.%S'),exact=False)

In [12]:
stats.head()

,Kills,Avg TTK,Score,Scenario,Date
0,2,0.0,0.0,180 Regen Warzone,2021-07-27 22:47:26
1,12,0.0,0.0,180 Regen Warzone,2021-07-27 22:48:41
2,7,0.0,0.0,180 Regen Warzone,2021-07-27 22:49:53
3,9,0.0,0.0,180 Regen Warzone,2021-07-28 22:45:19
4,10,0.0,0.0,180 Regen Warzone,2021-07-28 22:46:26


# SAVE

In [13]:
stats.to_csv("../data/stats.csv", index=False)

# 2. CREATE COUNTER THAT COUNTS UP MY HIGHESEST SCORING GAME IN REAL-TIME

### Search Bar

In [14]:
import difflib
scenario_names = (stats['Scenario'].unique()).astype(str)
user_input = '1wall 1target 10' # STREAMLIT -> user_input = st.text_input('Give me the name of a routine:')
closest_match = (difflib.get_close_matches(str(user_input), scenario_names, n=1,cutoff=0.45).pop(0))
closest_match

'1wall 1target - 10 Min'

### Date of highest score

In [15]:
scenario = stats.loc[stats['Scenario'] == closest_match]

In [16]:
import re

high_score_date = []

if sum(scenario['Score']) == 0:
    high_score_date.append(scenario
    .sort_values('Kills')
    ['Date'].iloc[-1])
elif scenario['Kills'] > 0 and sum(scenario['Score']) > 0:
    high_score_date.append(scenario
    .sort_values('Score')
    ['Date'].iloc[-1])

hs_date = str(high_score_date[0])
hs_date

'2021-08-18 18:11:32'

### Now we search for the .csv file with the highest score

In [17]:
hs_year = hs_date[:4]
hs_month = hs_date[5:7]
hs_day = hs_date[8:10]
hs_hour = hs_date[11:13]
hs_min = hs_date[14:16]
hs_sec = hs_date[17:19]

In [60]:
import re
import time

#def find_high_score_file():
Kill = []
Time = []
Acc = []

for filename in os.listdir(unzipped_stats):
    if f'{hs_year}.{hs_month}.{hs_day}-{hs_hour}.{hs_min}.{hs_sec} Stats.csv' in filename:
        f = open(f'{unzipped_stats}/{filename}', 'r')
        with f as csv_file:
            csv_reader = csv.reader(csv_file)
            for line in csv_reader:
                try:
                    Kill.append(line[0])
                    Time.append(line[1])
                    Acc.append(line[7])
                except Exception:
                    break 
        f.close()

In [98]:
data_tuples = list(zip(Kill,Time,Acc))

high_score_df = pd.DataFrame(data_tuples,columns=('Kill #','Time', 'Accuracy')).loc[1:]
high_score_df.head()

,Kill #,Time,Accuracy
1,1,18:01:33.001,1
2,2,18:01:34.055,0.5
3,3,18:01:35.103,1
4,4,18:01:35.755,1
5,5,18:01:36.526,1


CLEAN

In [107]:
high_score_df['Kills'] = pd.to_numeric(high_score_df['Kill #'], downcast='integer')
high_score_df['Accuracy'] = pd.to_numeric(high_score_df['Accuracy'], downcast='float')
high_score_df['Time'] = pd.to_datetime(high_score_df['Time'], format=('%H:%M:%S'),exact=False)

create counter.